In [2]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator ,array_to_img, img_to_array, load_img
from tensorflow.keras.models import load_model


emotion_dict = {0: "Angry", 1: "Disgusted", 2: "Fearful", 3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprised"}

# Model saved with Keras model.save()
MODEL_PATH = 'model.h5'

# Load your trained model
emotion_model = load_model(MODEL_PATH)


# start the webcam feed
#cap = cv2.VideoCapture(0)

# pass here your video path
cap = cv2.VideoCapture("What is a machine_ - Funny scene _ 3 Idiots _ Aamir Khan _ R Madhavan _ Sharman _HIGH.mp4")

while True:
    # Find haar cascade to draw bounding box around face
    ret, frame = cap.read()
    #frame = cv2.resize(frame, (1280, 720))
    if not ret:
        break
    face_detector = cv2.CascadeClassifier('haarcascades/haarcascade_frontalface_default.xml')
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # detect faces available on camera
    num_faces = face_detector.detectMultiScale(gray_frame, scaleFactor=1.3, minNeighbors=5)

    # take each face available on the camera and Preprocess it
    for (x, y, w, h) in num_faces:
        cv2.rectangle(frame, (x, y-50), (x+w, y+h+10), (0, 255, 0), 4)
        roi= gray_frame[y:y + h, x:x + w]
        roi = cv2.resize(roi,(48,48),interpolation=cv2.INTER_AREA) 
        roi = roi.astype('float')/255
        roi = img_to_array(roi)
        roi = np.expand_dims(roi, axis = 0)

        # predict the emotions
        emotion_prediction = emotion_model.predict(roi)
        maxindex = int(np.argmax(emotion_prediction))
        cv2.putText(frame, emotion_dict[maxindex], (x+5, y-20), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)

    cv2.imshow('Emotion Detection', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
